In [1]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pandas as pd
from pyspark.sql import SQLContext
conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)

sqlContext = SQLContext(sc)

In [2]:
df = spark.read.format("csv").option("header", "true").load("hoteis.csv")
columns_to_drop = ['endereco', 'telefone','site']
df = df.drop(*columns_to_drop)
df = df.limit(10)
df = df.selectExpr("_id as id", "nome as nome","latitude as latitude","longitude as longitude")
df.show()

#df.repartition(1).write.format('com.databricks.spark.csv').save("./vertex.csv",header = 'true')
df.toPandas().to_csv("vertex.csv")


+---+--------------------+---------+----------+
| id|                nome| latitude| longitude|
+---+--------------------+---------+----------+
|  1|Albergue Piratas ...|-8.107019|-34.889533|
|  2|   Boa Viagem Hostel|-8.110046|-34.894322|
|  3|Imperial Suítes Flat|-8.117141|-34.901665|
|  4|MercureAptMetrópolis|-8.064499|-34.894286|
|  5|MercureApt Navega...|-8.118182|-34.894833|
|  6|TulipInn Recife Flat|-8.139897|-34.905175|
|  7|Park Way Home Ser...|-8.123057|-34.901298|
|  8|       Atlante Plaza|-8.136259| -34.90194|
|  9|Best Western Mani...|-8.098362|-34.885681|
| 10|Boa Viagem Praia ...|-8.137725| -34.90228|
+---+--------------------+---------+----------+



In [3]:
df = df.selectExpr("id as _id", "nome as nome","latitude as latitude","longitude as longitude")
df.show()

+---+--------------------+---------+----------+
|_id|                nome| latitude| longitude|
+---+--------------------+---------+----------+
|  1|Albergue Piratas ...|-8.107019|-34.889533|
|  2|   Boa Viagem Hostel|-8.110046|-34.894322|
|  3|Imperial Suítes Flat|-8.117141|-34.901665|
|  4|MercureAptMetrópolis|-8.064499|-34.894286|
|  5|MercureApt Navega...|-8.118182|-34.894833|
|  6|TulipInn Recife Flat|-8.139897|-34.905175|
|  7|Park Way Home Ser...|-8.123057|-34.901298|
|  8|       Atlante Plaza|-8.136259| -34.90194|
|  9|Best Western Mani...|-8.098362|-34.885681|
| 10|Boa Viagem Praia ...|-8.137725| -34.90228|
+---+--------------------+---------+----------+



In [4]:
from math import radians, cos, sin, asin, sqrt 
def distance(obj1, obj2): 

    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(float(obj1[0]['longitude'])) 
    lon2 = radians(float(obj2[0]['longitude'])) 
    lat1 = radians(float(obj1[0]["latitude"])) 
    lat2 = radians(float(obj2[0]["latitude"])) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
    #print(c*r)
    # calculate the result 
    return(c * r) 
      
    

In [5]:
x = df.filter(df._id == 1)
x.head(3)

[Row(_id='1', nome='Albergue Piratas da Praia Hostel', latitude='-8.107019', longitude='-34.889533')]

In [6]:
edge_data= []
edge_data.append((1, 2, distance(df.filter(df._id == 1).head(1),df.filter(df._id == 2).head(1))))
edge_data.append((1, 3, distance(df.filter(df._id == 1).head(1),df.filter(df._id == 3).head(1))))
edge_data.append((1, 4, distance(df.filter(df._id == 1).head(1),df.filter(df._id == 4).head(1))))
edge_data.append((2, 6, distance(df.filter(df._id ==2).head(1),df.filter(df._id ==6).head(1))))
edge_data.append((2, 7, distance(df.filter(df._id ==2).head(1),df.filter(df._id ==7).head(1))))
edge_data.append((2, 4, distance(df.filter(df._id ==2).head(1),df.filter(df._id ==4).head(1))))
edge_data.append((3, 8, distance(df.filter(df._id ==3).head(1),df.filter(df._id ==8).head(1))))
edge_data.append((3, 9, distance(df.filter(df._id ==3).head(1),df.filter(df._id ==9).head(1))))
edge_data.append((3, 10, distance(df.filter(df._id ==3).head(1),df.filter(df._id ==10).head(1))))
edge_data.append((4, 5, distance(df.filter(df._id ==4).head(1),df.filter(df._id ==5).head(1))))
edge_data.append((4, 6, distance(df.filter(df._id ==4).head(1),df.filter(df._id ==6).head(1))))
edge_data.append((4, 7, distance(df.filter(df._id ==4).head(1),df.filter(df._id ==7).head(1))))
edge_data.append((5, 8, distance(df.filter(df._id ==5).head(1),df.filter(df._id ==8).head(1))))
edge_data.append((5, 9, distance(df.filter(df._id ==5).head(1),df.filter(df._id ==9).head(1))))
edge_data.append((5, 10, distance(df.filter(df._id ==5).head(1),df.filter(df._id ==10).head(1))))
edge_data.append((6, 3, distance(df.filter(df._id ==6).head(1),df.filter(df._id ==3).head(1))))
edge_data.append((6, 5, distance(df.filter(df._id ==6).head(1),df.filter(df._id ==5).head(1))))
edge_data.append((6, 7, distance(df.filter(df._id ==6).head(1),df.filter(df._id ==7).head(1))))
edge_data.append((7, 1, distance(df.filter(df._id ==7).head(1),df.filter(df._id ==1).head(1))))
edge_data.append((7, 8, distance(df.filter(df._id ==7).head(1),df.filter(df._id ==8).head(1))))
edge_data.append((7, 9, distance(df.filter(df._id ==7).head(1),df.filter(df._id ==9).head(1))))
edge_data.append((8, 1, distance(df.filter(df._id ==8).head(1),df.filter(df._id ==1).head(1))))
edge_data.append((8, 2, distance(df.filter(df._id ==8).head(1),df.filter(df._id ==2).head(1))))
edge_data.append((8, 4, distance(df.filter(df._id ==8).head(1),df.filter(df._id ==4).head(1))))
edge_data.append((9, 1, distance(df.filter(df._id ==9).head(1),df.filter(df._id ==1).head(1))))
edge_data.append((9, 2, distance(df.filter(df._id ==9).head(1),df.filter(df._id ==2).head(1))))
edge_data.append((9, 4, distance(df.filter(df._id ==9).head(1),df.filter(df._id ==4).head(1))))
edge_data.append((10, 2, distance(df.filter(df._id ==10).head(1),df.filter(df._id ==2).head(1))))
edge_data.append((10, 1, distance(df.filter(df._id ==10).head(1),df.filter(df._id ==1).head(1))))
edge_data.append((10, 4, distance(df.filter(df._id ==10).head(1),df.filter(df._id ==4).head(1))))
edge_data


[(1, 2, 0.6254749117398578),
 (1, 3, 1.7465376484835686),
 (1, 4, 4.7568748661268145),
 (2, 6, 3.527731569193673),
 (2, 7, 1.6379304279209408),
 (2, 4, 5.064596874547301),
 (3, 8, 2.1260401284606285),
 (3, 9, 2.730638602961474),
 (3, 10, 2.2898373199366535),
 (4, 5, 5.969580979785641),
 (4, 6, 8.469136928157475),
 (4, 7, 6.556949812805451),
 (5, 8, 2.156946277340633),
 (5, 9, 2.4232465350552195),
 (5, 10, 2.322558549974316),
 (6, 3, 2.559680473117969),
 (6, 5, 2.669511153829627),
 (6, 7, 1.9205395279301125),
 (7, 1, 2.2040016626020473),
 (7, 8, 1.4696954686115875),
 (7, 9, 3.2397228824032913),
 (8, 1, 3.5265430857778046),
 (8, 2, 3.0329862632132927),
 (8, 4, 8.023712282556527),
 (9, 1, 1.0518756739455306),
 (9, 2, 1.610211711494209),
 (9, 4, 3.882734236221232),
 (10, 2, 3.2000040115743023),
 (10, 1, 3.69143854402362),
 (10, 4, 8.189777770249627)]

In [7]:
edge = sqlContext.createDataFrame(edge_data).toDF("src", "dst", "relationship")
edge.describe().show()

edge.toPandas().to_csv("edges.csv")

+-------+------------------+-----------------+------------------+
|summary|               src|              dst|      relationship|
+-------+------------------+-----------------+------------------+
|  count|                30|               30|                30|
|   mean|               5.5|5.066666666666666| 3.422550539334681|
| stddev|2.9213837061606083|2.947042156343266|2.1134692156607646|
|    min|                 1|                1|0.6254749117398578|
|    max|                10|               10| 8.469136928157475|
+-------+------------------+-----------------+------------------+

